# Import libraries

In [1]:
import pandas as pd
import numpy as np
import mysql.connector

# Import conector
This didn't work due to problems with the connection with MySQL Server

# Importing Data and Cleaning
## Duration Table

In [2]:
#import table
duration_df = pd.read_csv("/home/inrx/Ironhack/Projects/Project-Week-2-Lisbon/dataset-lx-transp/IV.16.csv")

# rename the headers
duration_df = duration_df.rename(columns={"MUN":"Motive", "motor_bike":"motorbike","tax":"taxi"})

# Slice row table for work (the object of this analysis)
work_duration = duration_df[duration_df['Motive'] == 'Work']

# Drop the columns with the means of transportation we don't want to consider
work_duration = work_duration.drop(columns=['other','taxi','plane'])

# In order to determine the mode of transportation with max and min durations we decided 
# to transpose the DataFrame.
work_duration_t=work_duration.transpose()

# After transposing the DataFrame we had to rename the header with the correct name. This was done by 
# using the "rename" command to set the columns names has the first row (DataFrame.iloc[0])

work_duration_t = work_duration_t.rename(columns=work_duration_t.iloc[0])

# Afterwards we had to drop the row with the first row of the DataFrame
work_duration_t = work_duration_t.drop("Motive", axis=0)

# we also dedided to drop walking since we considerer not importante to our conclusions 
work_duration_t = work_duration_t.drop("walk", axis=0)

work_duration_t.head()

,Work
car_driver,24.8
car_pass,25.5
motorbike,19.5
public_bus,48.3
private_bus,33.2


In [3]:
work_duration_t.describe()

,Work
count,9.0
unique,9.0
top,19.5
freq,1.0


## Distance Table

In [4]:
#import
distance_df = pd.read_csv ("/home/inrx/Ironhack/Projects/Project-Week-2-Lisbon/dataset-lx-transp/IV.17.csv")
distance_df = distance_df.drop(columns=['plaine','other','táx'])
work_distance = distance_df[distance_df['Motive'] == 'Work']

# In order to determine the mode of transportation with max and min distances we decided 
# to transpose the DataFrame.
work_distance_t = work_distance.transpose()

# After transposing the DataFrame we had to rename the columns with the correct name. This was done by 
# using the "rename" command to set the columns names has the first row (DataFrame.iloc[0])

work_distance_t = work_distance_t.rename(columns=work_distance_t.iloc[0])

# Afterwards we had to drop the row with the first row of the DataFrame
work_distance_t = work_distance_t.drop("Motive", axis=0)

# we also dedided to drop walking since we considerer not importante to our conclusions 
work_distance_t = work_distance_t.drop("walk", axis=0)

work_distance_t.head()

,Work
car_driver,14.7
car_pass,15.8
motorbike,13.3
public_bus,12.5
private_bus,23.4


In [5]:
work_distance_t.describe()

,Work
count,9.0
unique,9.0
top,15.8
freq,1.0


## Layover Table

In [6]:
# importing table
layover_df = pd.read_csv ("/home/inrx/Ironhack/Projects/Project-Week-2-Lisbon/dataset-lx-transp/IV.18.csv", \
                          header=None, names=["transport","Total","0","1","> 2","a","b","c","d"])

# droping wrong columns 
layover_columns = ["a","b","c","d"]
layover_df = layover_df.drop(layover_columns, axis = 1)

# Finding the relative values per row for each column (percentage)

layover_df["0"]=layover_df["0"]*100/layover_df["Total"] 
layover_df["1"]=layover_df["1"]*100/layover_df["Total"] 
layover_df["> 2"]=layover_df["> 2"]*100/layover_df["Total"]
layover_df["Total"]=layover_df["Total"]*100/layover_df["Total"]

layover_df.head()

,transport,Total,0,1,> 2
0,car_driver,100.0,96.220762,3.565533,0.213705
1,car_pass,100.0,96.175160,3.424541,0.400301
2,motorbike,100.0,98.747164,1.221630,0.031206
3,taxi,100.0,94.813824,4.987164,0.199064
4,public_bus,100.0,55.334864,26.249279,18.415857


# Functions

In [8]:
def find_layovers_pct (transport):
    
    """ Returns the  number of layovers 
    for the desired transport (requires the name of the transport)"""
    
    row = layover_df.loc[layover_df['transport'] == transport].index.item()
    
    layovers_0 = round(layover_df.loc[row,'0'],2)
    layovers_1 = round (layover_df.loc[row,'1'],2)
    layovers_2 = round (layover_df.loc[row,'> 2'],2)
    
    max_pct_layover = max(layovers_0, layovers_1, layovers_2)    
    
    return max_pct_layover

In [9]:
def find_nmb_layover (transport):
    
    """Returns te number of layovers corresponding 
    to max layover per transport"""
    
    row = layover_df.loc[layover_df['transport'] == transport].index.item()
    
    layovers_0 = round(layover_df.loc[row,'0'],2)
    layovers_1 = round (layover_df.loc[row,'1'],2)
    layovers_2 = round (layover_df.loc[row,'> 2'],2)
    
    max_pct_layover = max(layovers_0, layovers_1, layovers_2)    
    
    nmb_layovers = lambda pct: 0 if pct == layovers_0 else \
    (1 if pct == layovers_1 else '> 2')
    
    return nmb_layovers(max_pct_layover)

In [37]:
def avg_speed (x,y):
    """Returns the average speed, given two arguments: x: distance (in km) and y: duration (in minutes)""" 
    return round(x/(y/60),2)

# Calculations

## Duration

In [10]:
"""
Find the average transport duration
"""

# calculating avg commute duration for work
mean_duration = round(work_duration_t["Work"].mean(),2)
print (f'The average duration of commute to go to work is {mean_duration} min.')

The average duration of commute to go to work is 36.68 min.


In [11]:
"""
Find the transport with longer duration
"""

# Find the modes of transportation with maximum duration 
max_duration = work_duration_t["Work"].max()

# Find the mode of transportation with max duration
max_duration_row = work_duration_t.loc[work_duration_t['Work'] == max_duration].index.item()

print (f'{max_duration_row.title()} is the transport with the longest duration, {max_duration} min.')

Boat is the transport with the longest duration, 54.2 min.


In [12]:
"""
Find the transport with shorter duration
"""

# Find the modes of transportation with min duration 
min_duration = work_duration_t["Work"].min()

# Find the mode of transportation with min duration
min_duration_row = work_duration_t.loc[work_duration_t['Work'] == min_duration].index.item()

print (f'{min_duration_row.title()} is the transport with the shorter duration, {min_duration} min.')

Motorbike is the transport with the shorter duration, 19.5 min.


## Commute distance

In [13]:
"""
Find the average transport distance
"""
mean_distance = round(work_distance_t["Work"].mean(),2)

print (f'The average distance of commute to work is {mean_distance} km.')

The average distance of commute to work is 14.9 km.


In [14]:
"""
Find the transport with shorter distance
"""

# Find the shortest distance in the table
min_distance = work_distance_t["Work"].min()

# Find the mode of transportation with min duration
min_distance_row = work_distance_t.loc[work_distance_t['Work'] == min_distance].index.item()

print (f'The transport that makes shorter distances is {min_distance_row.title()}, \
making {min_distance} km per day.')

The transport that makes shorter distances is Bike, making 8.0 km per day.


In [15]:
"""
Find the transport with longer distance
"""

# Find the longest distance in the table
max_distance = work_distance_t["Work"].max()

# Find the mode of transportation with max duration
max_distance_row = work_distance_t.loc[work_distance_t['Work'] == max_distance].index.item()

print (f'The transport that makes longer distances is {max_distance_row.title()}, \
making {max_distance} km per day.')

The transport that makes longer distances is Private_Bus, making 23.4 km per day.


### What if we exclude the Private Bus?

In [16]:
"""
Find the transport with longer distance, excluding private bus
"""

# work_distance_t_np = no private bus
work_distance_t_np = work_distance_t.drop("private_bus", axis=0)

# find the transport with bigger distance
max_distance_np = work_distance_t_np["Work"].max()

# find its name trhough index
max_distance_row_np = work_distance_t_np.loc[work_distance_t_np['Work'] == max_distance_np].index.item()

print (f'The transport that makes longer distances is {max_distance_row_np.title()}, \
making {max_distance_np} km per day.')

The transport that makes longer distances is Boat, making 19.0 km per day.


## Layover

In [17]:
"""
Find the transport with more layovers
"""

# find the max layovers row in the "> 2" column
max_layovers = layover_df["> 2"].max()

# Find the index corresponding to max_layovers
max_layovers_row = layover_df.loc[layover_df['> 2'] == max_layovers].index.item()

# Then, given the index, we find the transport of the corresponding index
max_layovers_transport = layover_df.loc[max_layovers_row,'transport'] #boat
max_layovers_transport = max_layovers_transport.lower()

print (f'The transport with the most layovers is {max_layovers_transport}, with \
about {round(max_layovers,2)} % of all people doing 2 or more layovers')

The transport with the most layovers is boat, with about 73.27 % of all people doing 2 or more layovers


In [32]:
"""
Find the transport with less layovers
"""

# find the min layovers row in the "0" column
min_layovers = layover_df["0"].max()

# Find the index coresponding to min_layovers
min_layovers_row = layover_df.loc[layover_df['0'] == min_layovers].index.item()

# Then, given the index, we find the transport of the corresponding index
min_layovers_transport = layover_df.loc[min_layovers_row,'transport']
min_layovers_transport = min_layovers_transport.lower()

print (f'The transport with the less layovers is {min_layovers_transport}, with \
about {round(min_layovers,2)} % of all people not doing any layovers.')

The transport with the less layovers is motorbike, with about 98.75 % of all people not doing any layovers.


## Relation between Duration and Distance

In [28]:
"""
Find the duration of the transport with the max and distance
"""

max_distance_duration = work_duration_t["Work"][max_distance_row]
min_distance_duration = work_duration_t["Work"][min_distance_row]

# Excluding private bus
max_distance_duration_np = work_duration_t["Work"][max_distance_row_np]

print (f'{max_distance_row.title()} is used, on average, for {max_distance_duration} min.')
print (f'{min_distance_row.title()} is used, on average, for {min_distance_duration} min.')
print (f'{max_distance_row_np.title()} is used, on average, for {max_distance_duration_np}.min')

Private_Bus is used, on average, for 33.2 min.
Bike is used, on average, for 32.2 min.
Boat is used, on average, for 54.2.min


In [20]:
"""
Find the distance of the transport with the max and duration
"""

max_duration_distance = work_distance_t["Work"][max_duration_row]
min_duration_distance = work_distance_t["Work"][min_duration_row]

print (f'{max_duration_row.title()} is used, on average, for {max_duration_distance} km.')
print (f'{min_duration_row.title()} is used, on average, for {min_duration_distance} km.')

Boat is used, on average, for 19.0 km.
Motorbike is used, on average, for 13.3 km.


## Finding the layovers for each transportation analysed above

In [29]:
"""
Find the number of layovers for the fastest and slowest transport
"""

min_dur_tpt_layovers_pct = find_layovers_pct(min_duration_row)
min_dur_tpt_layovers_nmb = find_nmb_layover(min_duration_row)

print (f'About {min_dur_tpt_layovers_pct} people traveling in {min_duration_row} \
make {min_dur_tpt_layovers_nmb} layovers.')

max_dur_tpt_layovers_pct = find_layovers_pct(max_duration_row)
max_dur_tpt_layovers_nmb = find_nmb_layover(max_duration_row)

print (f'About {max_dur_tpt_layovers_pct} people traveling in {max_duration_row} \
make {max_dur_tpt_layovers_nmb} layovers.')

About 98.75 people traveling in motorbike make 0 layovers.
About 73.27 people traveling in boat make > 2 layovers.


In [31]:
"""
Find the number of layovers for the longest and shorter transport
"""

min_dst_tpt_layovers_pct = find_layovers_pct(min_distance_row)
min_dst_tpt_layovers_nmb = find_nmb_layover(min_distance_row)

print (f'About {min_dst_tpt_layovers_pct} people traveling in {min_distance_row} \
make {min_dst_tpt_layovers_nmb} layovers.')

max_dst_tpt_layovers_pct = find_layovers_pct(max_distance_row)
max_dst_tpt_layovers_nmb = find_nmb_layover(max_distance_row)

print (f'About {max_dst_tpt_layovers_pct} people traveling in {max_distance_row} \
make {max_dst_tpt_layovers_nmb} layovers.')

max_dst_tpt_np_layovers_pct = find_layovers_pct(max_distance_row_np)
max_dst_tpt_np_layovers_nmb = find_nmb_layover(max_distance_row_np)

print (f'About {max_dst_tpt_np_layovers_pct} people traveling in {max_distance_row_np} \
make {max_dst_tpt_np_layovers_nmb} layovers.')

About 92.64 people traveling in bike make 0 layovers.
About 87.69 people traveling in private_bus make 0 layovers.
About 73.27 people traveling in boat make > 2 layovers.


## Speed

In [39]:
""" 
Find the average speed for the transports analysed above
"""

min_dur_speed = avg_speed(min_duration_distance, min_duration)
max_dur_speed = avg_speed(max_duration_distance, max_duration)
min_dst_speed = avg_speed(min_distance, min_distance_duration)
max_dst_speed = avg_speed(max_distance, max_distance_duration)
max_dst_speed_np = avg_speed(max_distance_np, max_distance_duration_np)


# Conclusions

## Does the number of layovers influences the duration of commute?
Our hypothesis was that the duration of the commute was influenced by the number of layovers 

In [26]:
if max_layovers_transport == max_duration_row and min_layovers_transport == min_duration_row:
    print (f'The number of layovers influences the commute duration.')
    
else:
    print (f"there is no relationship between the number of layovers and commute duration")

The number of layovers influences the commute duration.


## Are duration and distance positively related?
Our hypothesis was that the duration of the commute was related with the commute distance

### If we consider Private Bus:

In [21]:
if max_distance_row == max_duration_row:
    print ('Longer (time) commutes and longer distances are related')
    
else:
    print ('There is no relation between distance and duration of the commutes')

There is no relation between distance and duration of the commutes


### If we don't consider Private Bus:

In [22]:
if max_distance_row_np == max_duration_row:
    print ('Longer (time) commutes and longer distances are related')
    
else:
    print ('There is no relation between distance and duration of the commutes')

Longer (time) commutes and longer distances are related


# Sumary Table

In [58]:
conclusion_df = pd.DataFrame(columns = ["transport","km","duration","km/h","% layover","layover times"])

row_1 =[max_duration_row, max_duration_distance, max_duration, max_dur_speed,\
        max_dur_tpt_layovers_pct, max_dur_tpt_layovers_nmb]

row_2 = [min_duration_row, min_duration_distance, min_duration, min_dur_speed,\
        min_dur_tpt_layovers_pct, min_dur_tpt_layovers_nmb]

row_3 = [min_distance_row, min_distance, min_distance_duration, min_dst_speed,\
        min_dst_tpt_layovers_pct, min_dst_tpt_layovers_nmb]

row_4 = [max_distance_row, max_distance, max_distance_duration, max_dst_speed,\
        max_dst_tpt_layovers_pct, max_dst_tpt_layovers_nmb]

conclusion_df.loc[0] = row_1
conclusion_df.loc[1] = row_2
conclusion_df.loc[2] = row_3
conclusion_df.loc[3] = row_4

conclusion_df.sort_values("km/h", axis = 0, ascending = False,
                          inplace = True, na_position ='last') 

conclusion_df

,transport,km,duration,km/h,% layover,layover times
3,private_bus,23.4,33.2,42.29,87.69,0
1,motorbike,13.3,19.5,40.92,98.75,0
0,boat,19.0,54.2,21.03,73.27,> 2
2,bike,8.0,32.2,14.91,92.64,0
